In [1]:
from llm.llm.chatgpt import ChatGPT
from setup_db import create_chroma_db, connect_to_db, execute_query

In [2]:
llm = ChatGPT()

In [3]:
collection_chromadb = 'test_chroma_db'
persist_directory = 'data/test_chroma_db'
vector_store = create_chroma_db(collection_chromadb, persist_directory)

In [4]:
def similarity_search(query, k):
    return [f"'{document.page_content}'" for document in vector_store.similarity_search(query.strip("'"), k)]

In [5]:
from llm_test import text2sql, replace_category_query, CoT_reasoning

In [6]:
prompt = "What is the revenue and the profit of the companies with stock code TCB and VCB in first quarter of 2024"
code_generated = text2sql(llm, prompt)
code_generated

[{'language': 'sql',
  'code': "SELECT * FROM financial_statement fs\nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND category IN ('Revenue', 'Profit');\n"}]

In [7]:
for code in code_generated:
    if code['language'] == 'sql':
        code['code'] = replace_category_query(code['code'], similarity_search, top_k=5)
        # vector_store.execute_query(query)

In [8]:
test = "category = 'HYY'"
replace_category_query(test, similarity_search, top_k=5)

"category IN ('Attribute to parent company YoY (%)', 'Revenue YoY (%)', 'Attributable to parent company', 'Attribute to parent company (Bn. VND)', 'Interest and Similar Income')"

In [9]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
table_name = 'financial_statement'
port = '5433'
host = 'localhost'

In [10]:
code_generated

[{'language': 'sql',
  'code': "SELECT * FROM financial_statement fs\nWHERE ticker IN ('TCB', 'VCB') \nAND yearReport = 2024 \nAND lengthReport = 1 \nAND category IN ('Profit before tax', 'Total operating revenue', 'Operating Profit/Loss', 'Revenue YoY (%)', 'Net Sales', 'Sales', 'Revenue (Bn. VND)', 'Net Profit/Loss before tax', 'Gross Profit');\n"}]

In [12]:
conn = connect_to_db(db_name, user, password, host, port)
result = execute_query(code_generated[0]['code'], conn=conn, return_type='dataframe')
result

Connecting to database test_db, postgres...


,ticker,yearreport,lengthreport,category,data
0,TCB,2024,1,Revenue YoY (%),5.643908e-02
1,TCB,2024,1,Revenue (Bn. VND),1.450724e+13
2,TCB,2024,1,Total operating revenue,1.226177e+13
3,TCB,2024,1,Profit before tax,7.801659e+12
4,VCB,2024,1,Revenue YoY (%),-1.325470e-01
5,VCB,2024,1,Revenue (Bn. VND),2.401966e+13
6,VCB,2024,1,Total operating revenue,1.728040e+13
7,VCB,2024,1,Profit before tax,1.071824e+13


In [13]:
answer = CoT_reasoning(llm, prompt, result)
print(answer)

To answer the question of what is the revenue and profit of the companies with stock codes TCB and VCB for the first quarter of 2024, we can break down the available information systematically.

1. **Extract Information for TCB:**
   - Looking into the financial statement data provided:
     - For TCB (stock code TCB) in the first quarter of 2024, we find the following figures:
       - "Revenue (Bn. VND)": 14,507,238,000,000.0 VND
       - "Profit before tax": 7,801,659,000,000.0 VND
     
   - **Calculating TCB Revenue and Profit:**
     - Revenue: 14,507,238,000,000.0 VND
     - Profit: 7,801,659,000,000.0 VND

2. **Extract Information for VCB:**
   - Again, looking into the financial data:
     - For VCB (stock code VCB) in the first quarter of 2024, the relevant figures are:
       - "Revenue (Bn. VND)": 24,019,660,000,000.0 VND
       - "Profit before tax": 10,718,239,000,000.0 VND
     
   - **Calculating VCB Revenue and Profit:**
     - Revenue: 24,019,660,000,000.0 VND
     - 

category = xx

category IN (xx, yy)